In [ ]:
from langchain_ollama import ChatOllama

In [ ]:
import operator
from typing_extensions import Annotated, TypedDict, Literal
from pydantic import BaseModel, Field


base_llm = ChatOllama(
    model="codellama,
    temperature=0.0,
)


class RepoState(TypedDict):
    task: str
    code_snippets: Annotated[List[str], operator.add]
    test_status: Literal["Pass","Fail"]
    test_history: Annotated[List[str], operator.add]
    iterations: int

class EngineerState(TypedDict):
    recieved_code: str
    improved_code: str
    finished_improving: Literal["Finished","Unfinished"]

class TesterState(TypedDict):
    recieved_code: str
    pytest_code: str


class EngineerModel(BaseModel):
    recieved_code: str = Field(
        description="""
        This is the code that you have recieved and should attempt to improve by fixing typographical and logic issues.
        """
    )
    improved_code: str = Field(
        description="""
        This is the improved code which is returned by the llm.

        RULES
        - The code recieved should be improved so that the code returns is of a higher quality.
        - Output should contain only code correctly formatted.
        - Do not contain any conversational elements in the returned code.
        - Typographical and logic issues should have been fixed in this code.
        - If there are no improvements to be made then the recieved code should be returned as is.
        """
    )
    finished_improving: Literal["Finished","Unfinished"] = Field(
        description="""
        This is whether you have finished improving the code.

        RULES
        - If the improved code is the same as the recieved code then return Finished.
        - If the improved code is different from the revieved code then return Unfinished.
        """
    )
engineer_llm = base_llm.with_structured_output(EngineerModel)

class TesterModel(BaseModel):
    recieved_code: str = Field(
        description="""
        This is the code that you have recieved and should write a pytest file for to have maximum coverage.
        """
    )
    pytest_code: str = Field(
        description="""
        This is the written pytest code for the recieved code.

        RULES
        - The generated pytest code should only test what is present in the recieved code.
        - The returned pytest code should be correctly formatted.
        - The returned pytest code should have maximum coverage.
        """
    )
tester_llm = base_llm.with_structured_output(TesterModel)

In [ ]:
def create_poorly_formatted_code(state: State):
    result = base_llm.invoke("Write a short, correctly formatted code snippet, around 100 lines, with intentional inaccuracies, typographical errors and slight logic issues that a later llm could attempt to fix.")
    return {"code_iterations": [result.content]}

def supervisor(state: State):
    if state.get("code_snippets",None) is None:
        [Send(engineer_improvement, state["code_iterations"][-1])]
    elif state.get("test_status",None) == "Pass":
        [Send(test_finished_code, state[])]